In [1]:
from libraries.utility import Utility as mutil
from libraries.db_base import db_base
from libraries.db_ins_fake_data import fake_data_to_db
from libraries.custom_excel_output import custom_excel_output
import pandas as pd
import pandasql as psql
pd.options.display.max_columns = None
pd.set_option('display.max_colwidth', None)
pd.set_option('display.max_rows', None)
import warnings
import time
import os
import pandasql as psql
from IPython.core.display import Markdown as md
warnings.filterwarnings('ignore')

from IPython.core.display import display, HTML, Markdown, Latex
display(HTML(
    '<style>'
        '#notebook { padding-top:0px !important; } ' 
        '.container { width:100% !important; } '
        '.end_space { min-height:0px !important; } '
        '.end_space { min-height:0px !important; } '
        '.prompt {width: 0px; min-width: 0px; visibility: collapse } '
        '.parent{'
        '    display: grid;'
        '    grid-template-columns: 1fr 1fr;'
        '    column-gap: 5px;'
        '}    '
    '</style>'
))



In [2]:
dbprod = db_base("products")

Database : products, Connection Good: True


In [3]:
prod_info = dbprod.run_query_with_single_df(query_key="get_product_info")
prod_info.head(3)

,id,product_name,product_type
0,1,Pamela's Wild Ride,TV Series
1,2,Keith's Monster Madness,Movie
2,3,Kristin's Date in the City,Download


In [4]:
dbfin = db_base("finance",svr_type='postsql')

Database : finance, Connection Good: True


In [5]:
fin_account_activity = dbfin.run_query_with_single_df(query_key="get_account_activity")
fin_account_activity.head(3)

,id,product_id,customer_id,account_id,amt_usd,post_date,channel_desc,partner_desc,location_name,account_name,account_type
0,1,451,1,305,$23.86,2023-01-28 04:52:45,Museums and Cultural Institutions,Martin-Walters,Houston,Interest Income Revenue 115494,Revenue
1,2,146,1,354,$4.58,2023-05-19 19:11:58,Education,Freeman-Wall,Houston,Distribution Income Revenue 883265,Revenue
2,3,13,1,657,$19.55,2023-04-12 07:32:32,Home Video,"Scott, David and Jordan",Houston,Distribution Income Revenue 890265,Revenue


In [6]:
dbcust = db_base("customers",svr_type='mysql')

Database : customers, Connection Good: True


In [7]:
cust_products = dbcust.run_query_with_single_df(query_key="get_customer_product_history1")
cust_products.head(3)

,id,f_name,l_name,product_id,purchase_dt,expiration_dt,created_dt
0,1,Paul,Scott,146,2023-05-20 23:15:40,2024-01-09 19:11:58,2023-11-10 08:05:19
1,1,Paul,Scott,146,2023-05-19 19:11:58,2023-05-20 23:15:40,2023-11-10 08:05:19
2,1,Paul,Scott,13,2023-04-13 12:42:33,2023-04-14 07:32:32,2023-11-10 08:05:19


In [8]:
sql = '''

select * from cust_products where id = 3

'''
df = psql.sqldf(sql)
df.head()

,id,f_name,l_name,product_id,purchase_dt,expiration_dt,created_dt
0,3,Ricky,Montoya,186,2023-05-03 16:35:22.000000,2023-05-04 08:18:06.000000,2023-11-10 08:05:19.000000
1,3,Ricky,Montoya,186,2023-05-02 08:18:06.000000,2023-05-03 16:35:22.000000,2023-11-10 08:05:19.000000
2,3,Ricky,Montoya,224,2023-04-29 11:30:34.000000,2023-04-29 14:59:35.000000,2023-11-10 08:05:19.000000
3,3,Ricky,Montoya,224,2023-04-28 14:59:35.000000,2023-04-29 11:30:34.000000,2023-11-10 08:05:19.000000
4,3,Ricky,Montoya,193,2022-11-20 13:55:17.000000,2023-01-08 21:50:58.000000,2023-11-10 08:05:19.000000


In [9]:
fin_account_activity.head(1)

,id,product_id,customer_id,account_id,amt_usd,post_date,channel_desc,partner_desc,location_name,account_name,account_type
0,1,451,1,305,$23.86,2023-01-28 04:52:45,Museums and Cultural Institutions,Martin-Walters,Houston,Interest Income Revenue 115494,Revenue


In [10]:
display(md("### Join everthing together in pandassql"))
sql = '''
select
distinct
    cp.id as customer_id,
    cp.f_name,
    cp.l_name,
    p.product_name,
    p.product_type,
    fa.amt_usd,
    fa.post_date,
    fa.channel_desc,
    fa.partner_desc,
    fa.location_name,
    fa.account_name,
    fa.account_type
from cust_products as cp
join prod_info as p
on 
    p.id = cp.product_id
join fin_account_activity fa
on 
 fa.customer_id = cp.id
 and
 fa.product_id = cp.product_id
'''
df = psql.sqldf(sql)


### Join everthing together in pandassql

In [11]:
dbcust = custom_excel_output(current_database="customers",svr_type='mysql')

Database : customers, Connection Good: True


In [12]:
# Clean up the post_date column
df['post_date'] = pd.to_datetime(df['post_date'])
df['post_date'] = df['post_date'].dt.strftime('%Y-%m-%d %H:%M:%S')

u = mutil()
u.mkdir(os.path.join(u.get_this_dir(),"project_data","tempdir"))

display(md("### Example of Joined Data (Shuffled) and Excel Download"))

file_name = os.path.join(dbcust.get_this_dir(),"project_data","tempdir","customer_account_activity.xlsx")

df["amt_usd"] = df["amt_usd"].replace('[\$,]', '', regex=True).astype(float)

l_dfs = []
l_dfs.append(df)
dbcust.write_excel_from_dfs(list_of_dfs=l_dfs,file_name=file_name,add_subtotal_on_top=True)
href_tag = dbcust.get_embedded_href_tag_from_image_file(file_name)
dbcust.nukefile(file_name)
display(HTML(href_tag))
shuffled_df = df.sample(frac=1)
display(shuffled_df.head(5).style.set_properties(**{'text-align': 'left'}).set_table_styles([dict(selector = 'th', props=[('text-align', 'left')])]))

### Example of Joined Data (Shuffled) and Excel Download

,customer_id,f_name,l_name,product_name,product_type,amt_usd,post_date,channel_desc,partner_desc,location_name,account_name,account_type
6051,1123,Nicholas,Carr,Michael's Great War,Download,15.880000,2023-03-15 02:33:02,Education,"Scott, Brown and Gutierrez",Everett,Rental Income Revenue 912989,Revenue
580,105,Melvin,Phillips,Thomas's Trip in the Wild,Single View Movie,19.520000,2022-11-10 03:18:50,Music Publishing,"Phillips, Morgan and Carpenter",Chicago,Sell-Through Income Revenue 933441,Revenue
7074,1310,Christopher,Holmes,Mark's Wild Ride,Bundle,4.830000,2023-10-21 13:27:24,Home Video Rentals,Torres-Sanford,Norman,Interest Income Revenue 168592,Revenue
9532,1765,Susan,Thompson,Traci's Amazing Race,Bundle,33.360000,2023-07-04 22:53:09,Streaming Aggregators,Dixon-Baird,Sun Valley,Rental Income Revenue 730724,Revenue
11130,2060,Tammy,Hernandez,David's Trip in the Wild,Event,12.410000,2023-07-27 21:53:05,Theatrical,Reyes Ltd,Pickerington,Distribution Income Revenue 427454,Revenue


In [13]:
display(md("### Visualization of Key Data"))

import altair as alt
import pandas as pd

# Load your data here
data = df

# For the pie chart, visualize the revenue per product type
pie_data = data.groupby('product_type')['amt_usd'].sum().reset_index()

# For the donut chart, visualize the top 10 geographic locations by transaction count
top_locations = data['location_name'].value_counts().head(10).reset_index()
top_locations.columns = ['location_name', 'transactions']

# Define a more subdued color scheme
subdued_colors = ['#6baed6', '#bdd7e7', '#fdbe85', '#fdae6b', '#a1d99b', '#e5f5e0', '#bcbddc', '#9e9ac8', '#fcc5c0', '#fc9272']

# Creating a pie chart with a legend to the right and the subdued color scheme, formatting numbers in USD
pie_chart = alt.Chart(pie_data).mark_arc().encode(
    theta=alt.Theta(field='amt_usd', type='quantitative'),
    color=alt.Color(field='product_type', type='nominal', legend=alt.Legend(title='Product Type', orient='right'), scale=alt.Scale(range=subdued_colors)),
    tooltip=[alt.Tooltip('product_type:N'), alt.Tooltip('amt_usd:Q', title='Revenue', format='$,.2f')]
).properties(title='Revenue per Product Type', width=400)

# Creating a donut chart for the top 10 locations with a legend to the right and the subdued color scheme
donut_chart = alt.Chart(top_locations).mark_arc(innerRadius=50).encode(
    theta=alt.Theta(field='transactions', type='quantitative'),
    color=alt.Color(field='location_name', type='nominal', legend=alt.Legend(title='Top Locations', orient='right'), scale=alt.Scale(range=subdued_colors)),
    tooltip=[alt.Tooltip('location_name:N'), alt.Tooltip('transactions:Q', title='Transactions', format=',')]
).properties(title='Top 10 Geographic Locations by Transaction Count', width=400)


# Combine the charts
combined_charts = alt.hconcat(
    pie_chart, donut_chart
).resolve_scale(
    color='independent'
).configure_view(
    stroke=None
)


# Save the combined chart to an HTML file
combined_charts.save('combined_charts.html')

# If you also want to save them separately:
pie_chart.save('pie_chart.html')
donut_chart.save('donut_chart.html')

# To display the charts in the notebook, if you're running Jupyter Notebook or JupyterLab
combined_charts


### Visualization of Key Data

alt.HConcatChart(...)

In [15]:
import altair as alt

# ... [Your code for data loading and processing]

data = df

# Assuming 'data' is your DataFrame after cleaning
data['post_date'] = pd.to_datetime(data['post_date'])
data['amt_usd'] = pd.to_numeric(data['amt_usd'])

# Group the data by month and calculate the sum of 'amt_usd' in each month
monthly_revenue = data.groupby(data['post_date'].dt.to_period("M")).sum()['amt_usd'].reset_index()

# Convert the Period object to a string for 'post_date' column
monthly_revenue['post_date'] = monthly_revenue['post_date'].dt.strftime('%Y-%m')


# Altair Visualizations
# 1. Monthly Revenue Over Time
line = alt.Chart(monthly_revenue).mark_line().encode(
    x='post_date:T',
    y='amt_usd:Q'
)

points = alt.Chart(monthly_revenue).mark_point(size=150, opacity=0).encode(
    x='post_date:T',
    y='amt_usd:Q',
    tooltip=[alt.Tooltip('post_date:T', title='Date'), alt.Tooltip('amt_usd:Q', title='Revenue', format='$,.2f')]
)

monthly_revenue_chart = line + points
monthly_revenue_chart = monthly_revenue_chart.properties(
    title='Monthly Revenue Over Time',
    width=300,
    height=150
)
# 2. Distribution of Transaction Amounts with Sine Wave
transaction_distribution_chart = alt.Chart(data).mark_bar(color='#bdd7e7').encode(
    alt.X("amt_usd:Q", bin=alt.Bin(maxbins=30), title='Transaction Amount (USD)'),
    y='count()',
    tooltip=[alt.Tooltip('count()', title='Number of Transactions'), alt.Tooltip('amt_usd:Q', title='Amount', format='$,.2f')]
).properties(
    title='Distribution of Transaction Amounts',
    width=300,
    height=150
)  # Combine the sine wave chart with the transaction distribution chart

# Group the data by 'post_date' and 'product_type' and calculate the sum of 'amt_usd' in each group
revenue_product_type = data.groupby(['post_date', 'product_type']).sum()['amt_usd'].reset_index()

# Convert the 'post_date' column to a string for better visualization
revenue_product_type['post_date'] = revenue_product_type['post_date'].dt.strftime('%Y-%m')


# 3. Revenue by Product Type
revenue_product_chart = alt.Chart(revenue_product_type).mark_bar(color='#bdd7e7').encode(
    x='amt_usd:Q',
    y=alt.Y('product_type:N', sort='-x'),
    tooltip=[alt.Tooltip('product_type:N', title='Product Type'), alt.Tooltip('amt_usd:Q', title='Revenue', format='$,.2f')]
).properties(
    title='Revenue by Product Type',
    width=300,
    height=150
)

# Combine charts into a single visualization
combined_chart = alt.hconcat(monthly_revenue_chart, transaction_distribution_chart, revenue_product_chart).resolve_scale(
    color='independent'
)

alt.data_transformers.enable('json')

# Display the chart
combined_chart


alt.HConcatChart(...)

In [16]:
impor
# Define the pattern to match files (starting with 'altair-data' and ending with '.json')
pattern = 'altair-data*.json'

mu = mutil()

# Get a list of matching files in the directory
matching_files = glob.glob(os.path.join(mu.get_this_dir(), pattern))

# Loop through the matching files and delete them
for file_path in matching_files:
    os.remove(file_path)

NameError: name 'glob' is not defined